In [34]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from pprint import PrettyPrinter, pprint
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)

arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [3]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [5]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [6]:
wiki.invoke("What is machine Learning?")

'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\nML fi'

In [9]:
tavily = TavilySearchResults()

C:\Users\PMLS\AppData\Local\Temp\ipykernel_10716\2290291758.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [10]:
tavily.invoke("Provide me the recent AI news.")

[{'title': 'AI News | Latest AI News, Analysis & Events',
  'url': 'https://www.artificialintelligence-news.com/',
  'content': "Artificial Intelligence. August 1, 2025. Leak suggests OpenAI's open-source AI model release is imminent · Artificial Intelligence. August 1, 2025. Machine",
  'score': 0.69845337},
 {'title': 'AI News & Artificial Intelligence',
  'url': 'https://techcrunch.com/category/artificial-intelligence/',
  'content': '### OpenMind wants to be the Android operating system of humanoid robots\n\nThe Apple Siri and Apple Intelligence are being displayed on a smartphone screen and in the background.\n\n### Apple might be building its own AI ‘answer engine’\n\nOpenAI CEO Sam Altman speaks during the OpenAI DevDay event on November 06, 2023 in San Francisco, California.\n\n### Inside OpenAI’s quest to make AI do anything for you\n\n### The uproar over Vogue’s AI-generated ad isn’t just about fashion [...] ### ElevenLabs launches an AI music generator, which it claims is cl

## All tools

In [11]:
tools = [arxiv, wiki, tavily]

In [ ]:
llm = ChatGroq(model="Gemma2-9b-It")
llm_with_tools = llm.bind_tools(tools)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hg124wsgx', 'function': {'arguments': '{"query":"Pakistan Cricket Team Batting and Bowling Coach"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 1293, 'total_tokens': 1380, 'completion_time': 0.158181818, 'prompt_time': 0.034406168, 'queue_time': 0.295166449, 'total_time': 0.192587986}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--976229c2-30a2-4fcb-a927-8ff4502fe799-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Pakistan Cricket Team Batting and Bowling Coach'}, 'id': 'hg124wsgx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1293, 'output_tokens': 87, 'total_tokens': 1380})

In [19]:
llm_with_tools.invoke([
    HumanMessage(
        content=f"What is the recent AI news?"
    )
]).tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'recent AI news'},
  'id': '4ch9taq3t',
  'type': 'tool_call'}]

## Entire Chatbot with Langraph

#### State Definitions

In [22]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

#### Nodes Definition

In [23]:
def tool_calling_llm(state: State):
    
    return {
        "messages": [llm_with_tools.invoke(state['messages'])]
    }

#### Build Graph

In [24]:
builder = StateGraph(State)
builder.add_node('tool_calling_llm', tool_calling_llm)
builder.add_node('tools', ToolNode(tools))


builder.add_edge(START, 'tool_calling_llm')
builder.add_conditional_edges(
    'tool_calling_llm',
    tools_condition
)
builder.add_edge('tools', END)

In [30]:
graph = builder.compile()

#### View Graph

In [32]:
# visulaize the graph
from IPython.display import Image
Image(
    graph.get_graph().draw_mermaid_png()
)

KeyboardInterrupt: 

In [35]:
messages = graph.invoke({
    'messages': HumanMessage(
        content="What is attention is all you need?"
    )
})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is attention is all you need?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (35yxmk07j)
 Call ID: 35yxmk07j
  Args:
    query: Attention is all you need
================================= Tool Message =================================
Name: wikipedia

Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al. It is considered a foundational paper in modern artificial intelligence, and a main contributor to the AI boom, as the transformer approach has become the main architecture of a wi


In [36]:
messages = graph.invoke({
    'messages': HumanMessage(
        content="Provide me the top 10 recent new about AI."
    )
})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Provide me the top 10 recent new about AI.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (e2q21qb2j)
 Call ID: e2q21qb2j
  Args:
    query: top 10 recent news about AI
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Artificial Intelligence Tools Get Smarter", "url": "https://www.ift.org/news-and-publications/food-technology-magazine/issues/2025/august/features/artificial-intelligence--tools-get--smarter", "content": "Artificial Intelligence Tools Get Smarter August 5, 2025 · Dialogue · How Gen AI Will Accelerate Food Industry Innovation August 1, 2025.", "score": 0.98421}, {"title": "AI Update, August 1, 2025: AI News and Views From the ...", "url": "https://www.marketingprofs.com/opinions/2025/53515/ai-update-august-1-2025-ai-news-and-views-from-the-pa